# 1.8 Exploring the mesh topology

In this tutorial, we learn how to iterate over mesh entities and obtain information about how one mesh entity is connected to others. We will be able to answer questions like the following after this tutorial.

- What edges are connected to a vertex?
- What are the edges of an element?
- What are the points of an element? 
- What elements are adjacent to a face?

In some algorithms, such as custom block smoothers (that we shall see in [2.1.2](../unit-2.1.2-blockjacobi/blockjacobi.ipynb)), we will need to iterate over mesh entities using the concepts in this tutorial.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube

In [ ]:
mesh = Mesh(unit_cube.GenerateMesh(maxh=1))
Draw(mesh)

## Iterating over mesh objects

##### Iterating over vertices:

In [ ]:
for v in mesh.vertices:    
    print (v, v.point)

What is the type of `v` in `mesh.vertices`?

In [ ]:
for v in mesh.vertices:    
    print(type(v))   

We'll shortly return to this `MeshNode` class.

##### Iterating over elements:

In [ ]:
for el in mesh.Elements(VOL):
    print(type(el))
    print ("vertices: ", el.vertices)   # get vertices of an element 
    print ("edges: ", el.edges)         # get edges of an element 

### NodeId and MeshNodes

An object of type `NodeId` in NGSolve is just a number together with a type of the mesh entity it describes.


In [ ]:
v = NodeId(VERTEX,0)   # a standalone NodeId object
type(v)
print ("type = ", v.type, "v.nr =", v.nr)

`NODE_TYPE` can be one of the following:

- `VERTEX`: dimension 0 
- `EDGE`: dimension 1
- `FACE`: dimension 2
- `CELL`: dimension 3
- `ELEMENT`: codimension 0
- `FACET`: codimension 1 

E.g., in $n$ space dimensions, facets are mesh objects of dimension $n-1$. When $n=2$ there are no `CELL` entities.  

Nodes can be associated to an existing mesh. Consider the above-defined node 
```
v = NodeId(VERTEX,0)
```
When it is  associated to `mesh`, it becomes an object of type `MeshNode` which has coordinate information.

In [ ]:
meshv = mesh[v]
print ("type = ", type(meshv))
print ("point = ", meshv.point)

In [ ]:
type(v), type(meshv)   # note the different types

`MeshNode` objects like `meshv` can be queried for topology information. 

E.g.,  what are edges connected to the mesh vertex `meshv`?

In [ ]:
meshv.edges

In [ ]:
c = NodeId(CELL, 1)
meshc = mesh[c]
meshc.faces          # faces of a cell

In [ ]:
f = NodeId(FACE, 2)
meshf = mesh[f]
meshf.edges, meshf.elements


## ElementId and Ngs_Element

An `ElementId` is made using a number together with  an object like `BND` or `VOL` that knows the codimension.

In [ ]:
ei = ElementId(BND,0)
type(ei)

As with `NodeId`, we may associate `ElementId` with a mesh to get an object of type `Ngs_Element`, which can be queried for topology information.

In [ ]:
meshel = mesh[ei]
type(meshel)
print ("type of meshel = \n ", meshel)
print ("vertices =", meshel.vertices)

Note that `meshel` has only three vertices because it is a **boundary element**. 
Volume elements have the same type as boundary elements:

In [ ]:
elt0 = mesh[ElementId(VOL, 0)]
type(elt0)

In [ ]:
elt0.vertices, elt0.edges, elt0.facets

You can also access the same volume element using `NodeId`:

In [ ]:
el0 = mesh[NodeId(ELEMENT, 0)]
type(el0)

In [ ]:
el0.vertices, el0.edges

## Dofs

Dofs are numbers enumerating the global *degrees of freedom* of a finite element space. Dofs are associated to mesh entities of the previously described  types.

E.g., all dofs of the Lagrange finite element space associated to edges of the mesh can be obtained as follows.


In [ ]:
fes = H1(mesh, order=4)
for edge in mesh.edges:
    print ("type = ", type(edge)) 
    print ("dofs = ", fes.GetDofNrs(edge))   

From the output, it is evident that these edge dofs are associated to `MeshNode` objects.

What about dofs associated to elements?

In [ ]:
for el in fes.Elements(VOL):
    print(type(el))
    print (el.dofs)

The type `ngsolve.comp.FESpaceElement` appearing in the output is derived from `Ngs_Element` as can be seen from the documentation:

In [ ]:
from ngsolve.comp import FESpaceElement
help(FESpaceElement)